2.5 自动微分

In [1]:
import torch
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [2]:
x.requires_grad_(True)
x.grad

In [3]:
y = 2 * torch.dot(x, x) ## y 为两倍的 x和x的点积
y

tensor(28., grad_fn=<MulBackward0>)

In [4]:
y.backward() ## 反向传播给变量,此时应该是2x
x.grad

tensor([ 0.,  4.,  8., 12.])

In [5]:
x.grad == 4 * x ## 这是按原始比较？为什么结果是一个向量呢？

tensor([True, True, True, True])

In [13]:
x.grad.zero_() ## 清空梯度
y = x.sum() # y为x的每个元素之和 。相当于y = x的转置乘上E
y.backward()
x.grad

tensor([1., 1., 1., 1.])

In [18]:
x.grad.zero_()
y = x * x ## 按元素求乘积 因此y是个向量，每个元素都是x^2
##y.sum().backward() 
y.backward(torch.ones(len(x))) ## 这个和上面的y.sum().backward()是等价的。我目测是将y和传入的向量相乘得到标量，在进行backword
x.grad


tensor([ 0.,  4.,  8., 12.])

In [20]:
x.grad.zero_()
y = x * x
u = y.detach() ## 貌似是复制一个分离的y赋值给u
z = u * x   ## 这样z = u * x 其中，u看作一个常量，它的值是x * x
z.sum().backward()
x.grad == u


tensor([True, True, True, True])

In [21]:
x.grad.zero_()
y.sum().backward() ## y是x^2，因此它反向传播给x，则x.grad为2x
x.grad == 2 * x


tensor([True, True, True, True])

In [23]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 1000 * b
    return c

In [25]:
a = torch.rand(size=(), requires_grad=True)
d = f(a)
d.backward()

a.grad == d / a

tensor(True)

在上面f(a) 的结果一定是a的k倍，即相当于 f(a) = k * a
因此对f(a) 调用backword后，a.grad应该是 k 即:f(a) / a
